# Wines Points prediction 

In [3]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Here we will try to predict the points a wine will get based on known characteristics (i.e. features, in the ML terminology). The mine point in this stage is to establish a simple, ideally super cost effective, basline.
In the real world there is a tradeoff between complexity and perforamnce, and the DS job, among others, is to present a tradeoff tables of what performance is achivalbel at what complexity level. 

to which models with increased complexity and resource demands will be compared. Complexity should then be translated into cost. For example:
 * Compute cost 
 * Maintenance cost
 * Serving costs (i.e. is new platform needed?) 
 

## Loading the data

In [1]:
import pandas as pd
import cufflinks as cf; cf.go_offline()

In [2]:
wine_reviews = pd.read_csv("C:\\Users\\eranb\Python_ML_2023\\data\\winemag-data-130k-v2.csv")
wine_reviews.shape


(129971, 14)

In [64]:
#wine_reviews.head()

In [3]:
wine_reviews1=wine_reviews.drop(['taster_twitter_handle','title','region_1','region_2'], axis=1)


In [69]:
#wine_reviews1.points.value_counts().sort_index()

In [4]:
wine_reviews1.dropna(subset=['price'], inplace=True)
wine_reviews1.dropna(subset=['designation'], inplace=True)
wine_reviews1.dropna(subset=['province'], inplace=True)
wine_reviews1.dropna(subset=['variety'], inplace=True)
wine_reviews1.head()

,Unnamed: 0,country,description,designation,points,price,province,taster_name,variety,winery
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,Roger Voss,Portuguese Red,Quinta dos Avidagos
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Alexander Peartree,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Paul Gregutt,Pinot Noir,Sweet Cheeks
5,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Michael Schachner,Tempranillo-Merlot,Tandem
6,6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Kerin O’Keefe,Frappato,Terre di Giurfo


In [5]:
#no taster at all
wine_reviews12=wine_reviews1.drop(['taster_name'], axis=1)
wine_reviews12.shape


(86148, 9)

In [7]:
#no na taster 
wine_reviews11=wine_reviews1.dropna(subset=['taster_name'])

wine_reviews11.shape


(69919, 10)

In [9]:
wine_reviews11.isna().sum()

Unnamed: 0     0
country        0
description    0
designation    0
points         0
price          0
province       0
taster_name    0
variety        0
winery         0
dtype: int64

In [ ]:
#import pandas as pd
#import matplotlib.pyplot as plt

# Assuming the DataFrame is already loaded and named wine_reviews1

# Calculate the average points for each taster
#average_points = wine_reviews1.groupby('taster_name')['points'].mean()

# Create the graph
#plt.figure(figsize=(10, 6))
#average_points.plot(kind='bar')
#plt.xlabel('Taster Name')
#plt.ylabel('Average Points')
#plt.title('Average Points by Taster')
#plt.xticks(rotation=45)
#plt.ylim(80, 100)
#plt.show()

In [8]:
wine_reviews11.sample(5)

,Unnamed: 0,country,description,designation,points,price,province,taster_name,variety,winery
41793,41793,France,Ripe notes of juicy yellow-pear waft from the ...,W3,89,15.0,Alsace,Anne Krebiehl MW,Alsace white blend,Wolfberger
78551,78551,US,"Minty, camphor-like aromas arise off of this b...",Camp 4 Vineyard,87,40.0,California,Matt Kettmann,Cabernet Sauvignon,Kitá
106909,106909,Chile,Reduced gritty foresty aromas revolve around h...,Reserva Especial,87,19.0,Limarí Valley,Michael Schachner,Cabernet Franc,Viña Tabalí
51492,51492,Italy,"Espresso, roasted coffee bean, plum and toaste...",Prapò,91,110.0,Piedmont,Kerin O’Keefe,Nebbiolo,Ceretto
125,125,South Africa,Etienne Le Riche is a total Cabernet specialis...,Cabernet Sauvignon Reserve,91,45.0,Stellenbosch,Roger Voss,Cabernet Sauvignon,Le Riche


In [9]:
wine_reviews111=wine_reviews11.drop(['Unnamed: 0'], axis=1)

In [13]:
wine_reviews111.sample(3)

,country,description,designation,points,price,province,taster_name,variety,winery
7387,Australia,"The 2008 Brookman Syrah is a rich, plush, luxu...",Brookman,92,75.0,South Australia,Joe Czerwinski,Syrah,Clarendon Hills
84251,US,This is the producer's longstanding ode to his...,Orion Rossi Vineyard California Native,93,120.0,California,Virginie Boone,Red Blend,Sean Thackrey
125687,US,Carl Bowker makes some excellent wines from hi...,Kissin' Cousins,86,28.0,California,Matt Kettmann,Rhône-style White Blend,Caliza


In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Concatenate, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.utils import to_categorical, plot_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [14]:

data = wine_reviews111

X = data['description']
y = data['points']

max_words = 1000
tokenizer = Tokenizer(num_words=max_words, split=' ')
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
model = Sequential()
model.add(Embedding(max_words, 128, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(128, dropout=0.4, recurrent_dropout=0.4))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

epochs = 10
batch_size = 32
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

Epoch 1/10
1748/1748 [==============================] - 175s 99ms/step - loss: 835.5547 - mse: 835.5547 - val_loss: 8.8486 - val_mse: 8.8486
Epoch 2/10
1748/1748 [==============================] - 179s 102ms/step - loss: 8.8222 - mse: 8.8222 - val_loss: 8.8282 - val_mse: 8.8282
Epoch 3/10
1748/1748 [==============================] - 181s 103ms/step - loss: 8.3591 - mse: 8.3591 - val_loss: 7.8219 - val_mse: 7.8219
Epoch 4/10
1748/1748 [==============================] - 182s 104ms/step - loss: 7.4777 - mse: 7.4777 - val_loss: 6.8746 - val_mse: 6.8746
Epoch 5/10
1748/1748 [==============================] - 182s 104ms/step - loss: 4.5624 - mse: 4.5624 - val_loss: 3.7932 - val_mse: 3.7932
Epoch 6/10
1748/1748 [==============================] - 183s 105ms/step - loss: 3.7016 - mse: 3.7016 - val_loss: 3.5061 - val_mse: 3.5061
Epoch 7/10
1748/1748 [==============================] - 182s 104ms/step - loss: 3.5376 - mse: 3.5376 - val_loss: 4.1595 - val_mse: 4.1595
Epoch 8/10
1748/1748 [=========

In [17]:
y_pred = model.predict(X_test)


print(y_pred)

437/437 [==============================] - 7s 16ms/step
[[89.95739]
 [86.25712]
 [88.70486]
 ...
 [89.96043]
 [92.00674]
 [89.11213]]


In [18]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(y_test, y_pred)

rmse = np.sqrt(mse)

mae = mean_absolute_error(y_test, y_pred)

print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)

MSE: 3.29362335378406
RMSE: 1.8148342496724212
MAE: 1.425987304211754


In [ ]:
#MSE: 3.29362335378406
#RMSE: 1.8148342496724212
#MAE: 1.425987304211754

In [20]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Concatenate, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.utils import to_categorical, plot_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

data = wine_reviews111
description_max_length = 100  
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['description'])
sequences = tokenizer.texts_to_sequences(data['description'])
description_data = pad_sequences(sequences, maxlen=description_max_length)
target = data['points']

label_encoders = {}
categorical_columns = ['country', 'designation', 'province', 'taster_name', 'variety', 'winery']
for column in categorical_columns:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

categorical_data = data[categorical_columns].values

X_train_desc, X_test_desc, X_train_cat, X_test_cat, y_train, y_test = train_test_split(
    description_data, categorical_data, target, test_size=0.2, random_state=42)

embedding_dim = 100
vocab_size = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=description_max_length))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

model_cat = Sequential()
model_cat.add(Dense(16, activation='relu', input_shape=(len(categorical_columns),)))

combined_model = Concatenate()([model.output, model_cat.output])
combined_model = Dense(32, activation='relu')(combined_model)
combined_model = Dense(1, activation='linear')(combined_model)

final_model = Model(inputs=[model.input, model_cat.input], outputs=combined_model)

final_model.compile(loss='mean_squared_error', optimizer='adam')
final_model.fit([X_train_desc, X_train_cat], y_train, epochs=10, batch_size=32, validation_split=0.2)

loss = final_model.evaluate([X_test_desc, X_test_cat], y_test)
print('Mean Squared Error:', loss)


Epoch 1/10
1399/1399 [==============================] - 42s 30ms/step - loss: 11413.2842 - val_loss: 39.5631
Epoch 2/10
1399/1399 [==============================] - 42s 30ms/step - loss: 32.7621 - val_loss: 38.0451
Epoch 3/10
1399/1399 [==============================] - 41s 29ms/step - loss: 33.0036 - val_loss: 180.8522
Epoch 4/10
1399/1399 [==============================] - 41s 30ms/step - loss: 53.3116 - val_loss: 9.1825
Epoch 5/10
1399/1399 [==============================] - 42s 30ms/step - loss: 12.6884 - val_loss: 10.5878
Epoch 6/10
1399/1399 [==============================] - 41s 30ms/step - loss: 29.9118 - val_loss: 25.4585
Epoch 7/10
1399/1399 [==============================] - 41s 29ms/step - loss: 21.1248 - val_loss: 85.5089
Epoch 8/10
1399/1399 [==============================] - 42s 30ms/step - loss: 27.5867 - val_loss: 6.3963
Epoch 9/10
1399/1399 [==============================] - 41s 30ms/step - loss: 16.8163 - val_loss: 6.1779
Epoch 10/10
437/437 [========================